In [1]:
!pip install torchcodec
!pip install datasets pyarrow==16.0.0 --quiet
!pip install soundfile

import datasets
import torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login

HF_TOKEN = "..."

login(token=HF_TOKEN)

In [3]:
# Setup completo para Google Colab
!pip install -q transformers accelerate bitsandbytes
!git clone https://github.com/kehanlu/DeSTA2.git
%cd DeSTA2
!pip install -e .

# Parche para el bug
import transformers.models.bert.modeling_bert as bert_modeling

original_init = bert_modeling.BertAttention.__init__

def patched_init(self, config, *args, **kwargs):
    if hasattr(config, '_attn_implementation'):
        if config._attn_implementation is None or config._attn_implementation == "torch":
            config._attn_implementation = 'eager'
    return original_init(self, config, *args, **kwargs)

bert_modeling.BertAttention.__init__ = patched_init

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.2 MB/s eta 0:00:00
Cloning into 'DeSTA2'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 86 (delta 23), reused 72 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (86/86), 4.54 MiB | 18.96 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/DeSTA2
Obtaining file:///content/DeSTA2
  Preparing metadata (setup.py) ... done
  Running setup.py develop for desta


In [4]:
# Cargar modelo
from desta import DestaModel
import torch

model = DestaModel.from_pretrained(
    "DeSTA-ntu/DeSTA2-8B-beta",
    load_in_8bit=True,
    device_map="auto"
)

print("✅ Ready to use!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

qformer_connector.pth:   0%|          | 0.00/89.0M [00:00<?, ?B/s]

✅ Ready to use!


In [5]:
from IPython.display import Audio
import numpy as np
import soundfile as sf
import os

# Cargar el dataset
dataset_name = "DynamicSuperb/NoiseSNRLevelPrediction_VCTK_MUSAN-Music"
data = datasets.load_dataset(dataset_name)

# Ver la estructura primero
print("Splits disponibles:", list(data.keys()))
print("Tables:", data[list(data.keys())[0]].column_names)

# Acceder al split de prueba
split_name = list(data.keys())[0]  # Usar el primer split, que es 'test'
dataset_split = data[split_name]

# Directorio para guardar los audios (opcional, pero útil para organizar)
audio_output_dir = "./processed_audios"
os.makedirs(audio_output_dir, exist_ok=True)

print(f"\nProcessing {len(dataset_split)} examples...")

for idx, example in enumerate(dataset_split):
    print(f"\n--- Processing example {idx} ---")
    print(f"Complete example information: {example}")

    if 'audio' in example:
        audio_data = example['audio']

        # Crear un nombre de archivo único para cada audio
        audio_filename = f"audio_{idx}.wav"
        full_output_path = os.path.join(audio_output_dir, audio_filename)

        try:
            sf.write(full_output_path, audio_data['array'], audio_data['sampling_rate'])
            print(f"✅ Audio saved to: {full_output_path}")
        except Exception as e:
            print(f"❌ Error saving audio for example {idx}: {e}")

        # Si deseas reproducir en el notebook para cada uno (puede ser mucho)
        # display(Audio(audio_data['array'], rate=audio_data['sampling_rate']))


README.md:   0%|          | 0.00/628 [00:00<?, ?B/s]

data/test-00000-of-00001-3a97b4f888e0173(…):   0%|          | 0.00/25.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

Splits disponibles: ['test']
Tables: ['file', 'audio', 'instruction', 'label']

Processing 200 examples...

--- Processing example 0 ---
Complete example information: {'file': 'clean_VCTK-Corpus_16k_p239_502.wav', 'audio': <datasets.features._torchcodec.AudioDecoder object at 0x7bdcbabf9a00>, 'instruction': 'Identify the SNR of the utterance affected by music noise. The answer could be zero, five, ten, fifteen, or clean.', 'label': 'clean'}
✅ Audio saved to: ./processed_audios/audio_0.wav

--- Processing example 1 ---
Complete example information: {'file': 'music_snr15_VCTK-Corpus_16k_p250_173.wav', 'audio': <datasets.features._torchcodec.AudioDecoder object at 0x7bdcb4eb0800>, 'instruction': 'State the SNR Signal-to-Noise Ratio for the utterance with music noise. The answer could be zero, five, ten, fifteen, or clean.', 'label': 'fifteen'}
✅ Audio saved to: ./processed_audios/audio_1.wav

--- Processing example 2 ---
Complete example information: {'file': 'music_snr15_VCTK-Corpus_16k_

In [ ]:
"""# Los archivos de ejemplo están en /content/DeSTA2/assets/audios/
audio_path = "/content/DeSTA2/assets/audios/7_1_d7.wav"

messages = [
    {"role": "system", "content": "Focus on the audio clip."},
    {"role": "audio", "content": audio_path},
    {"role": "user", "content": "Describe the audio clip."},
]

print("🔄 Generando respuesta...")
generated_ids = model.chat(messages, max_new_tokens=256, do_sample=True, temperature=0.6, top_p=0.9)
response = model.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("\n✅ Respuesta:")
print(response)"""

'# Los archivos de ejemplo están en /content/DeSTA2/assets/audios/\naudio_path = "/content/DeSTA2/assets/audios/7_1_d7.wav"\n\nmessages = [\n    {"role": "system", "content": "Focus on the audio clip."},\n    {"role": "audio", "content": audio_path},\n    {"role": "user", "content": "Describe the audio clip."},\n]\n\nprint("🔄 Generando respuesta...")\ngenerated_ids = model.chat(messages, max_new_tokens=256, do_sample=True, temperature=0.6, top_p=0.9)\nresponse = model.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]\n\nprint("\n✅ Respuesta:")\nprint(response)'

In [ ]:
generated_labels = []
true_labels = []

print("✅ Initialized 'generated_labels' and 'true_labels' lists.")

In [ ]:
instructions = []
audio_paths = []
true_labels_collected = []

for idx, example in enumerate(data['test']):
    instruction = example['instruction']
    true_label = example['label']
    audio_filename = f"audio_{idx}.wav"
    full_audio_path = os.path.join(audio_output_dir, audio_filename)

    instructions.append(instruction)
    audio_paths.append(full_audio_path)
    true_labels_collected.append(true_label)

print(f"✅ Collected data for {len(instructions)} examples.")

In [ ]:
for i in range(len(instructions)):
    instruction_text = instructions[i]
    audio_file_path = audio_paths[i]
    true_label_val = true_labels_collected[i]

    messages = [
        {"role": "system", "content": "You are an expert in audio analysis. Extract the SNR from the given audio in the following format: zero, five, ten, fifteen, or clean."},
        {"role": "audio", "content": audio_file_path},
        {"role": "user", "content": instruction_text},
    ]

    try:
        # Generate prediction
        generated_ids = model.chat(messages, max_new_tokens=256, do_sample=True, temperature=0.6, top_p=0.9)
        response = model.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Extract the predicted label from the response
        # This assumes the model's response will contain one of the target labels
        predicted_label = "unknown"
        possible_labels = ['zero', 'five', 'ten', 'fifteen', 'clean']
        for label_word in possible_labels:
            if label_word in response.lower():
                predicted_label = label_word
                break

        generated_labels.append(predicted_label)
        true_labels.append(true_label_val)

        if i % 20 == 0:
            print(f"Processed {i+1}/{len(instructions)} examples. Last true: {true_label_val}, predicted: {predicted_label}")

    except Exception as e:
        print(f"Error processing example {i}: {e}")
        generated_labels.append("error") # Mark as error if prediction fails
        true_labels.append(true_label_val) # Still append true label for consistency

print("\n✅ All predictions generated.")
print(f"Total generated labels: {len(generated_labels)}")
print(f"Total true labels: {len(true_labels)}")

## Accuarcy NoiseSNRLevelPrediction_VCTK_MUSAN-Music

In [ ]:
# Calculate accuracy
correct_predictions = 0
for i in range(len(true_labels)):
    if generated_labels[i] == true_labels[i]:
        correct_predictions += 1

total_examples = len(true_labels)
accuracy = correct_predictions / total_examples if total_examples > 0 else 0

print(f"\nTotal correct predictions: {correct_predictions}")
print(f"Overall Accuracy: {accuracy:.4f}")


Total correct predictions: 40
Overall Accuracy: 0.2000


In [ ]:
for i in range(len(true_labels)):
  print(f'Generated label: {generated_labels[i]} and true label: {true_labels[i]}')


Generated label: five and true label: clean
Generated label: five and true label: fifteen
Generated label: five and true label: fifteen
Generated label: five and true label: clean
Generated label: unknown and true label: clean
Generated label: unknown and true label: clean
Generated label: five and true label: clean
Generated label: five and true label: ten
Generated label: five and true label: fifteen
Generated label: five and true label: clean
Generated label: unknown and true label: five
Generated label: five and true label: five
Generated label: five and true label: clean
Generated label: ten and true label: zero
Generated label: five and true label: ten
Generated label: five and true label: zero
Generated label: clean and true label: clean
Generated label: five and true label: clean
Generated label: ten and true label: zero
Generated label: five and true label: zero
Generated label: ten and true label: clean
Generated label: five and true label: five
Generated label: ten and true 

20 Prozent sind so gut wie ein Zufallsgenerator der random einer der fünf möglichen Outputs wählt. Miserables Ergebnis also.

In [ ]:
speaker_verification_data = datasets.load_dataset("DynamicSuperb/SpeakerVerification_LibriSpeech-TestClean")

print('Splits disponibles:', list(speaker_verification_data.keys()))

first_split = speaker_verification_data[list(speaker_verification_data.keys())[0]]
print('Tables:', first_split.column_names)

README.md:   0%|          | 0.00/648 [00:00<?, ?B/s]

data/test-00000-of-00001-c35c45ec9ffb8c9(…):   0%|          | 0.00/48.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

Splits disponibles: ['test']
Tables: ['file', 'audio', 'file2', 'audio2', 'instruction', 'label']


In [ ]:
audio_output_dir_sv = "./processed_audios_speaker_verification"
os.makedirs(audio_output_dir_sv, exist_ok=True)

speaker_verification_instructions = []
speaker_verification_audio_paths_1 = []
speaker_verification_audio_paths_2 = []
speaker_verification_true_labels = []

split_name_sv = list(speaker_verification_data.keys())[0]
dataset_split_sv = speaker_verification_data[split_name_sv]

print(f"\nProcessing {len(dataset_split_sv)} examples from Speaker Verification dataset...")

for idx, example in enumerate(dataset_split_sv):
    instruction = example['instruction']
    true_label = example['label']

    # Process first audio file
    if 'audio' in example:
        audio_data_1 = example['audio']
        audio_filename_1 = f"sv_audio_1_{idx}.wav"
        full_output_path_1 = os.path.join(audio_output_dir_sv, audio_filename_1)
        try:
            sf.write(full_output_path_1, audio_data_1['array'], audio_data_1['sampling_rate'])
        except Exception as e:
            print(f"❌ Error saving first audio for example {idx}: {e}")

    # Process second audio file
    if 'audio2' in example:
        audio_data_2 = example['audio2']
        audio_filename_2 = f"sv_audio_2_{idx}.wav"
        full_output_path_2 = os.path.join(audio_output_dir_sv, audio_filename_2)
        try:
            sf.write(full_output_path_2, audio_data_2['array'], audio_data_2['sampling_rate'])
        except Exception as e:
            print(f"❌ Error saving second audio for example {idx}: {e}")

    speaker_verification_instructions.append(instruction)
    speaker_verification_audio_paths_1.append(full_output_path_1)
    speaker_verification_audio_paths_2.append(full_output_path_2)
    speaker_verification_true_labels.append(true_label)

    if idx % 50 == 0:
        print(f"Processed {idx+1}/{len(dataset_split_sv)} speaker verification examples.")

print("✅ All speaker verification audio files and data processed and saved.")


Processing 200 examples from Speaker Verification dataset...
Processed 1/200 speaker verification examples.
Processed 51/200 speaker verification examples.
Processed 101/200 speaker verification examples.
Processed 151/200 speaker verification examples.
✅ All speaker verification audio files and data processed and saved.


In [ ]:
sv_generated_labels = []
sv_true_labels = []

print("✅ Initialized 'sv_generated_labels' and 'sv_true_labels' lists for speaker verification.")

✅ Initialized 'sv_generated_labels' and 'sv_true_labels' lists for speaker verification.


In [ ]:
for i in range(len(speaker_verification_instructions)):
    instruction_text = speaker_verification_instructions[i]
    audio_file_path_1 = speaker_verification_audio_paths_1[i]
    audio_file_path_2 = speaker_verification_audio_paths_2[i]
    true_label_val = speaker_verification_true_labels[i]

    messages = [
        {"role": "system", "content": "You are an expert in speaker verification. Based on the two provided audio samples, determine if they belong to the same speaker. The answer should be either 'yes' or 'no'."},
        {"role": "audio", "content": audio_file_path_1},
        {"role": "audio2", "content": audio_file_path_2},
        {"role": "user", "content": instruction_text},
    ]

    try:
        generated_ids = model.chat(messages, max_new_tokens=256, do_sample=False, temperature=0.0)
        response = model.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Extract the predicted label (yes/no) from the response
        predicted_label = "unknown"
        if "yes" in response.lower():
            predicted_label = "yes"
        elif "no" in response.lower():
            predicted_label = "no"

        sv_generated_labels.append(predicted_label)
        sv_true_labels.append(true_label_val)

        if i % 20 == 0:
            print(f"Processed {i+1}/{len(speaker_verification_instructions)} SV examples. Last true: {true_label_val}, predicted: {predicted_label}")

    except Exception as e:
        print(f"Error processing speaker verification example {i}: {e}")
        sv_generated_labels.append("error") # Mark as error if prediction fails
        sv_true_labels.append(true_label_val) # Still append true label for consistency

print("\n✅ All speaker verification predictions generated.")
print(f"Total generated SV labels: {len(sv_generated_labels)}")
print(f"Total true SV labels: {len(sv_true_labels)}")

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=i

Processed 1/200 SV examples. Last true: no, predicted: no
Processed 21/200 SV examples. Last true: no, predicted: no
Processed 41/200 SV examples. Last true: no, predicted: no
Processed 61/200 SV examples. Last true: no, predicted: no
Processed 81/200 SV examples. Last true: no, predicted: no
Processed 101/200 SV examples. Last true: no, predicted: no
Processed 121/200 SV examples. Last true: yes, predicted: no
Processed 141/200 SV examples. Last true: yes, predicted: no
Processed 161/200 SV examples. Last true: yes, predicted: yes
Processed 181/200 SV examples. Last true: yes, predicted: no

✅ All speaker verification predictions generated.
Total generated SV labels: 200
Total true SV labels: 200


In [ ]:
correct_predictions_sv = 0
for i in range(len(sv_true_labels)):
    if sv_generated_labels[i] == sv_true_labels[i]:
        correct_predictions_sv += 1

total_examples_sv = len(sv_true_labels)
accuracy_sv = correct_predictions_sv / total_examples_sv if total_examples_sv > 0 else 0

print(f"\nTotal correct speaker verification predictions: {correct_predictions_sv}")
print(f"Overall Speaker Verification Accuracy: {accuracy_sv:.4f}")


Total correct speaker verification predictions: 87
Overall Speaker Verification Accuracy: 0.4350


In [ ]:
for i in range(len(sv_true_labels)):
  print(f'Generated label: {sv_generated_labels[i]} and true label: {sv_true_labels[i]}')


Generated label: no and true label: no
Generated label: yes and true label: no
Generated label: yes and true label: no
Generated label: no and true label: no
Generated label: no and true label: no
Generated label: yes and true label: no
Generated label: no and true label: no
Generated label: no and true label: no
Generated label: no and true label: no
Generated label: no and true label: no
Generated label: no and true label: no
Generated label: unknown and true label: no
Generated label: yes and true label: no
Generated label: unknown and true label: no
Generated label: yes and true label: no
Generated label: yes and true label: no
Generated label: yes and true label: no
Generated label: no and true label: no
Generated label: unknown and true label: no
Generated label: no and true label: no
Generated label: no and true label: no
Generated label: no and true label: no
Generated label: yes and true label: no
Generated label: yes and true label: no
Generated label: unknown and true label:

## Summary:

The accuracy of 0.43 is worse than random choice of 0.5.


In [ ]:
speaker_verification_other_data = datasets.load_dataset("DynamicSuperb/SpeakerVerification_LibriSpeech-TestOther")

print('Splits disponibles (TestOther):', list(speaker_verification_other_data.keys()))

first_split_other = speaker_verification_other_data[list(speaker_verification_other_data.keys())[0]]
print('Tables (TestOther):', first_split_other.column_names)

README.md:   0%|          | 0.00/418 [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Splits disponibles (TestOther): ['test']
Tables (TestOther): ['file', 'audio', 'audio2', 'instruction', 'label']


In [ ]:
audio_output_dir_sv_other = "./processed_audios_speaker_verification_other"
os.makedirs(audio_output_dir_sv_other, exist_ok=True)

speaker_verification_other_instructions = []
speaker_verification_other_audio_paths_1 = []
speaker_verification_other_audio_paths_2 = []
speaker_verification_other_true_labels = []

split_name_sv_other = list(speaker_verification_other_data.keys())[0]
dataset_split_sv_other = speaker_verification_other_data[split_name_sv_other]

print(f"\nProcessing {len(dataset_split_sv_other)} examples from Speaker Verification (TestOther) dataset...")

for idx, example in enumerate(dataset_split_sv_other):
    instruction = example['instruction']
    true_label = example['label']

    # Process first audio file
    if 'audio' in example:
        audio_data_1 = example['audio']
        audio_filename_1 = f"sv_other_audio_1_{idx}.wav"
        full_output_path_1 = os.path.join(audio_output_dir_sv_other, audio_filename_1)
        try:
            sf.write(full_output_path_1, audio_data_1['array'], audio_data_1['sampling_rate'])
        except Exception as e:
            print(f"❌ Error saving first audio for example {idx}: {e}")

    # Process second audio file
    if 'audio2' in example:
        audio_data_2 = example['audio2']
        audio_filename_2 = f"sv_other_audio_2_{idx}.wav"
        full_output_path_2 = os.path.join(audio_output_dir_sv_other, audio_filename_2)
        try:
            sf.write(full_output_path_2, audio_data_2['array'], audio_data_2['sampling_rate'])
        except Exception as e:
            print(f"❌ Error saving second audio for example {idx}: {e}")

    speaker_verification_other_instructions.append(instruction)
    speaker_verification_other_audio_paths_1.append(full_output_path_1)
    speaker_verification_other_audio_paths_2.append(full_output_path_2)
    speaker_verification_other_true_labels.append(true_label)

    if idx % 50 == 0:
        print(f"Processed {idx+1}/{len(dataset_split_sv_other)} speaker verification (TestOther) examples.")

print("✅ All speaker verification (TestOther) audio files and data processed and saved.")


Processing 100 examples from Speaker Verification (TestOther) dataset...
Processed 1/100 speaker verification (TestOther) examples.
Processed 51/100 speaker verification (TestOther) examples.
✅ All speaker verification (TestOther) audio files and data processed and saved.


In [ ]:
sv_other_generated_labels = []
sv_other_true_labels = []

print("✅ Initialized 'sv_other_generated_labels' and 'sv_other_true_labels' lists for speaker verification (TestOther).")

✅ Initialized 'sv_other_generated_labels' and 'sv_other_true_labels' lists for speaker verification (TestOther).


In [ ]:
for i in range(len(speaker_verification_other_instructions)):
    instruction_text = speaker_verification_other_instructions[i]
    audio_file_path_1 = speaker_verification_other_audio_paths_1[i]
    audio_file_path_2 = speaker_verification_other_audio_paths_2[i]
    true_label_val = speaker_verification_other_true_labels[i]

    messages = [
        {"role": "system", "content": "You are an expert in speaker verification. Based on the two provided audio samples, determine if they belong to the same speaker. The answer should be either 'yes' or 'no'."},
        {"role": "audio", "content": audio_file_path_1},
        {"role": "audio2", "content": audio_file_path_2},
        {"role": "user", "content": instruction_text},
    ]

    try:
        generated_ids = model.chat(messages, max_new_tokens=256, do_sample=False, temperature=0.0)
        response = model.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Extract the predicted label (yes/no) from the response
        predicted_label = "unknown"
        if "yes" in response.lower():
            predicted_label = "yes"
        elif "no" in response.lower():
            predicted_label = "no"

        sv_other_generated_labels.append(predicted_label)
        sv_other_true_labels.append(true_label_val.lower()) # Ensure labels are lowercase for comparison

        if i % 20 == 0:
            print(f"Processed {i+1}/{len(speaker_verification_other_instructions)} SV (TestOther) examples. Last true: {true_label_val}, predicted: {predicted_label}")

    except Exception as e:
        print(f"Error processing speaker verification (TestOther) example {i}: {e}")
        sv_other_generated_labels.append("error") # Mark as error if prediction fails
        sv_other_true_labels.append(true_label_val.lower()) # Still append true label for consistency

print("\n✅ All speaker verification (TestOther) predictions generated.")
print(f"Total generated SV (TestOther) labels: {len(sv_other_generated_labels)}")
print(f"Total true SV (TestOther) labels: {len(sv_other_true_labels)}")

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=i

Processed 1/100 SV (TestOther) examples. Last true: No, predicted: no
Processed 21/100 SV (TestOther) examples. Last true: No, predicted: unknown
Processed 41/100 SV (TestOther) examples. Last true: No, predicted: unknown
Processed 61/100 SV (TestOther) examples. Last true: Yes, predicted: no
Processed 81/100 SV (TestOther) examples. Last true: Yes, predicted: unknown

✅ All speaker verification (TestOther) predictions generated.
Total generated SV (TestOther) labels: 100
Total true SV (TestOther) labels: 100


In [ ]:
correct_predictions_sv_other = 0
for i in range(len(sv_other_true_labels)):
    if sv_other_generated_labels[i].lower() == sv_other_true_labels[i].lower():
        correct_predictions_sv_other += 1

total_examples_sv_other = len(sv_other_true_labels)
accuracy_sv_other = correct_predictions_sv_other / total_examples_sv_other if total_examples_sv_other > 0 else 0

print(f"\nTotal correct speaker verification (TestOther) predictions: {correct_predictions_sv_other}")
print(f"Overall Speaker Verification (TestOther) Accuracy: {accuracy_sv_other:.4f}")


Total correct speaker verification (TestOther) predictions: 44
Overall Speaker Verification (TestOther) Accuracy: 0.4400


In [ ]:
for i in range(len(sv_other_true_labels)):
  print(f'Generated label: {sv_other_generated_labels[i]} and true label: {sv_other_true_labels[i]}')


Generated label: no and true label: no
Generated label: no and true label: no
Generated label: unknown and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: no
Generated label: unknown and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: no
Generated label: no and true label: no
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: no
Generated label: no and true label: yes
Generated label: no and true label: no
Generated label: no and true label: no
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: yes and true label: yes
Generated label: unknown and true label: no
Generated label: no and true label: no
Generated label: no and true label: yes
Generated label: no and true label: no
Generated label: no and true label:

0.44 is worse than random choice of 0.5

## Summary:

### Q&A
1.  **What is the accuracy for the `DynamicSuperb/SpeakerVerification_LibriSpeech-TestOther` dataset?**
2.  **How many correct predictions were made?**

### Data Analysis Key Findings

### Insights or Next Steps

In [6]:
speaker_verification_vctk_data = datasets.load_dataset("DynamicSuperb/SpeakerVerification_VCTK")

print('Splits disponibles (VCTK):', list(speaker_verification_vctk_data.keys()))

first_split_vctk = speaker_verification_vctk_data[list(speaker_verification_vctk_data.keys())[0]]
print('Tables (VCTK):', first_split_vctk.column_names)

README.md:   0%|          | 0.00/632 [00:00<?, ?B/s]

data/test-00000-of-00001-6c3437893991a2d(…):   0%|          | 0.00/69.9M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

Splits disponibles (VCTK): ['test']
Tables (VCTK): ['file', 'audio', 'file2', 'audio2', 'instruction', 'label']


In [7]:
audio_output_dir_sv_vctk = "./processed_audios_speaker_verification_vctk"
os.makedirs(audio_output_dir_sv_vctk, exist_ok=True)

speaker_verification_vctk_instructions = []
speaker_verification_vctk_audio_paths_1 = []
speaker_verification_vctk_audio_paths_2 = []
speaker_verification_vctk_true_labels = []

split_name_sv_vctk = list(speaker_verification_vctk_data.keys())[0]
dataset_split_sv_vctk = speaker_verification_vctk_data[split_name_sv_vctk]

print(f"\nProcessing {len(dataset_split_sv_vctk)} examples from Speaker Verification (VCTK) dataset...")

for idx, example in enumerate(dataset_split_sv_vctk):
    instruction = example['instruction']
    true_label = example['label']

    # Process first audio file
    if 'audio' in example:
        audio_data_1 = example['audio']
        audio_filename_1 = f"sv_vctk_audio_1_{idx}.wav"
        full_output_path_1 = os.path.join(audio_output_dir_sv_vctk, audio_filename_1)
        try:
            sf.write(full_output_path_1, audio_data_1['array'], audio_data_1['sampling_rate'])
        except Exception as e:
            print(f"❌ Error saving first audio for example {idx}: {e}")

    # Process second audio file
    if 'audio2' in example:
        audio_data_2 = example['audio2']
        audio_filename_2 = f"sv_vctk_audio_2_{idx}.wav"
        full_output_path_2 = os.path.join(audio_output_dir_sv_vctk, audio_filename_2)
        try:
            sf.write(full_output_path_2, audio_data_2['array'], audio_data_2['sampling_rate'])
        except Exception as e:
            print(f"❌ Error saving second audio for example {idx}: {e}")

    speaker_verification_vctk_instructions.append(instruction)
    speaker_verification_vctk_audio_paths_1.append(full_output_path_1)
    speaker_verification_vctk_audio_paths_2.append(full_output_path_2)
    speaker_verification_vctk_true_labels.append(true_label)

    if idx % 50 == 0:
        print(f"Processed {idx+1}/{len(dataset_split_sv_vctk)} speaker verification (VCTK) examples.")

print("✅ All speaker verification (VCTK) audio files and data processed and saved.")


Processing 200 examples from Speaker Verification (VCTK) dataset...
Processed 1/200 speaker verification (VCTK) examples.
Processed 51/200 speaker verification (VCTK) examples.
Processed 101/200 speaker verification (VCTK) examples.
Processed 151/200 speaker verification (VCTK) examples.
✅ All speaker verification (VCTK) audio files and data processed and saved.


In [8]:
sv_vctk_generated_labels = []
sv_vctk_true_labels = []

print("✅ Initialized 'sv_vctk_generated_labels' and 'sv_vctk_true_labels' lists for speaker verification (VCTK).")

✅ Initialized 'sv_vctk_generated_labels' and 'sv_vctk_true_labels' lists for speaker verification (VCTK).


In [9]:
for i in range(len(speaker_verification_vctk_instructions)):
    instruction_text = speaker_verification_vctk_instructions[i]
    audio_file_path_1 = speaker_verification_vctk_audio_paths_1[i]
    audio_file_path_2 = speaker_verification_vctk_audio_paths_2[i]
    true_label_val = speaker_verification_vctk_true_labels[i]

    messages = [
        {"role": "system", "content": "You are an expert in speaker verification. Based on the two provided audio samples, determine if they belong to the same speaker. The answer should be either 'yes' or 'no'."},
        {"role": "audio", "content": audio_file_path_1},
        {"role": "audio2", "content": audio_file_path_2},
        {"role": "user", "content": instruction_text},
    ]

    try:
        generated_ids = model.chat(messages, max_new_tokens=256, do_sample=False, temperature=0.0)
        response = model.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Extract the predicted label (yes/no) from the response
        predicted_label = "unknown"
        if "yes" in response.lower():
            predicted_label = "yes"
        elif "no" in response.lower():
            predicted_label = "no"

        sv_vctk_generated_labels.append(predicted_label)
        sv_vctk_true_labels.append(true_label_val.lower()) # Ensure labels are lowercase for comparison

        if i % 20 == 0:
            print(f"Processed {i+1}/{len(speaker_verification_vctk_instructions)} SV (VCTK) examples. Last true: {true_label_val}, predicted: {predicted_label}")

    except Exception as e:
        print(f"Error processing speaker verification (VCTK) example {i}: {e}")
        sv_vctk_generated_labels.append("error") # Mark as error if prediction fails
        sv_vctk_true_labels.append(true_label_val.lower()) # Still append true label for consistency

print("\n✅ All speaker verification (VCTK) predictions generated.")
print(f"Total generated SV (VCTK) labels: {len(sv_vctk_generated_labels)}")
print(f"Total true SV (VCTK) labels: {len(sv_vctk_true_labels)}")

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=i

Processed 1/200 SV (VCTK) examples. Last true: yes, predicted: no
Processed 21/200 SV (VCTK) examples. Last true: yes, predicted: no
Processed 41/200 SV (VCTK) examples. Last true: yes, predicted: no
Processed 61/200 SV (VCTK) examples. Last true: yes, predicted: no
Processed 81/200 SV (VCTK) examples. Last true: yes, predicted: no
Processed 101/200 SV (VCTK) examples. Last true: yes, predicted: no
Processed 121/200 SV (VCTK) examples. Last true: no, predicted: yes
Processed 141/200 SV (VCTK) examples. Last true: no, predicted: yes
Processed 161/200 SV (VCTK) examples. Last true: no, predicted: yes
Processed 181/200 SV (VCTK) examples. Last true: no, predicted: no

✅ All speaker verification (VCTK) predictions generated.
Total generated SV (VCTK) labels: 200
Total true SV (VCTK) labels: 200


In [10]:
correct_predictions_sv_vctk = 0
for i in range(len(sv_vctk_true_labels)):
    if sv_vctk_generated_labels[i].lower() == sv_vctk_true_labels[i].lower():
        correct_predictions_sv_vctk += 1

total_examples_sv_vctk = len(sv_vctk_true_labels)
accuracy_sv_vctk = correct_predictions_sv_vctk / total_examples_sv_vctk if total_examples_sv_vctk > 0 else 0

print(f"\nTotal correct speaker verification (VCTK) predictions: {correct_predictions_sv_vctk}")
print(f"Overall Speaker Verification (VCTK) Accuracy: {accuracy_sv_vctk:.4f}")


Total correct speaker verification (VCTK) predictions: 89
Overall Speaker Verification (VCTK) Accuracy: 0.4450


In [11]:
for i in range(len(sv_vctk_true_labels)):
  print(f'Generated label: {sv_vctk_generated_labels[i]} and true label: {sv_vctk_true_labels[i]}')


Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: yes and true label: yes
Generated label: unknown and true label: yes
Generated label: no and true label: yes
Generated label: yes and true label: yes
Generated label: yes and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: unknown and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and true label: yes
Generated label: no and tru